In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
print("tensorflow version: ", tf.__version__)

# Name of dataset in tensorflow datasets
DATASET_NAME="rock_you"

2021-10-09 11:04:11.339634: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-09 11:04:11.339672: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


tensorflow version:  2.6.0


Load RockYou Data

In [3]:
dataset, dataset_info = tfds.load(name=DATASET_NAME,
                                  split='train[:75%]',
                                  with_info=True)

2021-10-09 11:04:12.555225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-09 11:04:12.555262: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-09 11:04:12.555280: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hafez-HP-EliteDesk-800-G1-SFF): /proc/driver/nvidia/version does not exist
2021-10-09 11:04:12.555516: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
print(dataset_info)
dataset = dataset.shuffle(100000, reshuffle_each_iteration=True)

# Cache dataset for future use
dataset = dataset.cache()


tfds.core.DatasetInfo(
    name='rock_you',
    full_name='rock_you/1.0.0',
    description="""
    This dataset contains 14,344,391 passwords that were leaked or stolen from
    various sites. The author of this dataset states that "I'm hosting them because
    it seems like nobody else does (hopefully it isn't because hosting them is
    illegal :)). Naturally, I'm not the one who stole these; I simply found them
    online, removed any names/email addresses/etc.".
    
    This dataset is used to train Machine Learning models for password guessing
    and cracking.
    """,
    homepage='https://wiki.skullsecurity.org/Passwords',
    data_path='/home/hafez/tensorflow_datasets/rock_you/1.0.0',
    download_size=133.44 MiB,
    dataset_size=393.36 MiB,
    features=FeaturesDict({
        'password': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=None,
    disable_shuffling=False,
    splits={
        'train': <SplitInfo num_examples=14344391, num_shards=8>,
    },
    ci

Build Generator Network based on the architecture in the paper


In [ ]:
# create a class Residual block based on Residual Networks definition
class ResidualBlock(tf.keras.Model):
    def __init__(self, dim):
        super().__init__()
        self.res_block = tf.keras.Sequential([
            tf.keras.layers.ReLU(True),
            tf.keras.layers.Conv1D(dim, dim, 5, padding='same'),
            tf.keras.layers.ReLU(True),
            tf.keras.layers.Conv1D(dim, dim, 5, padding='same'),
        ])

    def call(self, input_data, **kwargs):
        output = self.res_block(input_data)
        return input_data + (0.3 * output)

# create a class Generator Network
class GeneratorNetwork(tf.keras.Model):
    def __init__(self, dim):
        self.dim = dim

        # instantiate a Sequential Model
        self.generator_res_block_model = tf.keras.models.Sequential()

        # first linear layer
        self.first_linear_layer = tf.keras.layers.Dense()

        # residual blocks in a sequential order
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))
        self.generator_res_block_model.add(ResidualBlock(dim=5))

        # convolutional 1D layer
        self.conv_1d_layer = tf.keras.layers.Conv1D()

        # last soft max layer
        self.softmax_layer = tf.keras.layers.Softmax()

    def call(self, input_noise, **kwargs):
        """

        :param input_noise: noise input of some sample generated passwords
        :param kwargs:
        :return: the generated passwords for an iteration
        """

        # feed first layer with noise data
        generated_output = self.first_linear_layer(input_noise)

        # reshape the result of linear layer
        generated_output = tf.reshape(generated_output)

        # feed residual blocks by output from reshape stage
        generated_output = self.generator_res_block_model(generated_output)

        # feed resulted data to convolutional layer
        generated_output = self.conv_1d_layer(generated_output)

        # transpose operation on the resulted output
        generated_output = tf.transpose(generated_output)

        # feed softmax layer with transposed output
        generated_output = self.softmax_layer(generated_output)

        return generated_output